# Setup

Initial module setup.

In [1]:
import typing
import time
import statistics
import numpy.typing as np_types
import pandas as pd

from concrete.ml.sklearn.xgb import XGBClassifier
from eeg_auth_models_framework import data, pre_process, features, training, model, processor
from eeg_auth_models_framework.utils import conversion

/home/experiment/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Constants

In [2]:
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0.5
K_FOLDS = 10
RANDOM_SEED = 42

# Model Builder Configuration

Configure data source, data reading method, data labelling method, and training process. 

In [3]:
class XGBBuilder(model.ModelBuilder[XGBClassifier]):    
    def create_classifier(self) -> XGBClassifier:
        # see: https://github.com/Rahul2K20/EEG_Authentication/blob/main/Main.ipynb
        return XGBClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.3,
            random_state=RANDOM_SEED
        )
    
    def train_classifier(self, classifier: XGBClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike):
        classifier.fit(x_data, y_data)
        
    def score_classifier(self, classifier: XGBClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike) -> float:
        return classifier.score(x_data, y_data)

# Configuration

In [4]:
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
labeller = training.SubjectDataLabeller()
converter_filter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)
converter_psd = conversion.MNEDataFrameConverter(sample_frequency=DATASET_SAMPLE_FREQ_HZ)

# Data Processing

## Pre-Processing Steps

Define pre-processing steps to be used in model.

In [5]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES, 
        converter_filter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

## Feature Extraction Steps

Define feature extraction steps to be applied to the pre-processed data.

In [6]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.PSDExtractor(converter_psd)
])

## Data Processor

In [7]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps
)

# Training

Execute training of authentication models.

In [8]:
xgb_builder = XGBBuilder(
    data_downloader=downloader,
    data_reader=reader,
    data_labeller=labeller,
    data_processor=data_processor
)
results = xgb_builder.train(K_FOLDS)

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
  

In [9]:
subject_score_rows = []
for subject in results.training_statistics:
    subject_score_rows.append([
        subject,
        results.training_statistics[subject].average_score,
        results.training_statistics[subject].training_duration
    ])
subject_scores = pd.DataFrame(subject_score_rows, columns=['Subject', 'Average Score', 'Training Duration'])
subject_scores

,Subject,Average Score,Training Duration
0,S17,0.987639,344.690686
1,S12,0.939444,347.117352
2,S18,0.982670,356.699021
3,S02,0.975154,357.582885
4,S01,0.959151,350.368204
5,S05,0.962932,356.289837
6,S07,0.981404,365.800732
7,S19,0.990093,357.482743
8,S08,0.981373,361.314263
9,S20,0.985108,345.040936


In [10]:
subject_rates_rows = []
for subject in results.training_statistics:
    subject_rates_rows.append([
        subject,
        results.training_statistics[subject].average_false_accept_rate,
        results.training_statistics[subject].average_false_reject_rate
    ])
subject_rates = pd.DataFrame(subject_rates_rows, columns=['Subject', 'Average FAR', 'Average FRR'])
subject_rates

,Subject,Average FAR,Average FRR
0,S17,0.013004,0.000000
1,S12,0.058459,0.100000
2,S18,0.005195,0.250000
3,S02,0.001316,0.483333
4,S01,0.027307,0.325000
5,S05,0.024324,0.188095
6,S07,0.001299,0.358333
7,S19,0.005212,0.100000
8,S08,0.003913,0.300000
9,S20,0.009091,0.125000


In [11]:
global_stats = pd.DataFrame(
    [[results.global_average_score, results.global_average_far_rate, 
      results.global_average_frr_rate, results.global_average_time]],
    columns=['Global Average Score', 'Global Average FAR', 'Global Average FRR', 'Global Average Time']
)
global_stats

,Global Average Score,Global Average FAR,Global Average FRR,Global Average Time
0,0.969406,0.287321,0.017265,354.333946


# Simulated Execution

In [12]:
T = typing.TypeVar("T")

def slice_by_percentage(content: typing.List[T], percentage: float) -> typing.List[T]:
    slice_size = int(len(content) * percentage / 100)
    return content[:slice_size]


def simulate_model_executions(models: typing.Dict[str, XGBClassifier], 
                              test_downloader: data.AuditoryDataDownloader, 
                              test_reader: data.AuditoryDataReader,
                              test_processor: processor.DataProcessor,
                              tests = 10) -> typing.Tuple[int, float, float]:
    data_path = test_downloader.retrieve()
    test_data = test_reader.format_data(data_path)
    for subject, data_samples in test_data.items():
        test_data[subject] = test_processor.process(data_samples)
    compile_timings = []
    predict_timings = []
    for i in range(tests):
        for subject, prepared_data in test_data.items():
            compile_set = slice_by_percentage(prepared_data, 20)
            test_model = models[subject]
            start_compile = time.perf_counter()
            test_model.compile(compile_set)
            end_compile = time.perf_counter()
            start_predict = time.perf_counter()
            test_model.predict(prepared_data, fhe="execute")
            end_predict = time.perf_counter()
            compile_timings.append(end_compile - start_compile)
            predict_timings.append(end_predict - start_predict)
    return len(predict_timings), statistics.mean(compile_timings), statistics.mean(predict_timings)

In [13]:
executions, average_compile_time, average_predict_time = simulate_model_executions(results.models, downloader, reader, data_processor, tests=1)
print(f"Executions: {executions}")
print(f"Overall average compile time: {average_compile_time} seconds")
print(f"Overall average predict time (encrypted): {average_predict_time} seconds")

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
  